In [1]:
# do basic imports and unpack McMurdo data

from pmagpy import ipmag
reload(ipmag)
from pmagpy import pmag
from programs import new_builder as nb
from programs import data_model3
reload(data_model3)
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
from programs.new_builder import Contribution

import pmagpy.controlled_vocabularies3 as cv



/Users/nebula/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


got full_df
got parsed_df
got full_df
got parsed_df


In [2]:
dir_path = os.path.join(os.getcwd(), '3_0', 'Megiddo')
con = Contribution(dir_path)

loc_dm = con.tables['locations'].data_model.dm['locations']
loc_df = con.tables['locations'].df
site_dm = con.tables['sites'].data_model.dm['sites']
site_df = con.tables['sites'].df
samp_df = con.tables['samples'].df
samp_dm = con.tables['samples'].data_model.dm['samples']
spec_df = con.tables['specimens'].df
spec_dm = con.tables['specimens'].data_model.dm['specimens']
age_df = con.tables['ages'].df
age_dm = con.tables['ages'].data_model.dm['ages']
meas_df = con.tables['measurements'].df
meas_dm = con.tables['measurements'].data_model.dm['measurements']
cont_df = con.tables['contribution'].df
cont_dm = con.tables['contribution'].data_model.dm['contribution']
crit_df = con.tables['criteria'].df
crit_dm = con.tables['criteria'].data_model.dm['criteria']


current_con = con

# mess up some validations for locations
loc_df.loc['Tel Hazor', 'lat_s'] = 400.
loc_df['dir_inc'] = 5
loc_df.loc['Tel Hazor', 'lat_n'] = 'hello'
loc_df.loc[:, 'lithologies'] = ["Agate", "random"]
#current_con.tables.pop('sites')

# mess up some validations for sites
#site_df.pop('age')
#site_df['dir_tilt_correction'] = 1
#site_df['dir_tilt_correction'] = 'one'

# mess up some validations for samples
samp_df.pop('citations')
samp_df.iloc[0].lon = 600.
samp_df.iloc[1].lat = 'hello'
samp_df.iloc[2].specimens = "hz05a2:fake"
samp_df.iloc[3].specimens = "fake : hz05a1"
samp_df.iloc[5].specimens = 'fake_specimen'
samp_df.iloc[7].site = 'fake_site'

# mess up some validations for measurements
meas_df.loc['mgh05a01:LP-PI-TRM1', 'magn_moment'] = 2
meas_df.loc['mgh05a01:LP-PI-TRM1', 'specimen'] = "fake_specimen"
meas_df.pop('experiment')

#current_df.head()

-W- No such file: /Users/nebula/Python/PmagPy/3_0/Megiddo/images.txt


measurement
mgh05a01:LP-PI-TRM0     mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM1     mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM2     mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM3     mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM4     mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM5     mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM6     mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM7     mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM8     mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM9     mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM10    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM11    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM12    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM13    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM14    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM15    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM16    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM17    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM18    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM19    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM20    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM21    mgh05a01:LP-PI-TRM
mgh05a01:LP-PI-TRM22    mgh05a01:LP-PI-TRM

In [3]:
import pmagpy.controlled_vocabularies3 as cv
reload(cv)
vocab = cv.Vocabulary()
vocabulary, possible_vocabulary = vocab.get_controlled_vocabularies()


-I- Importing controlled vocabularies from https://earthref.org


In [4]:



# need to add requiredOneInGroup

def requiredUnless(col_name, df, arg, *args):
    arg_list = arg.split(",")
    arg_list = [arg.strip('"') for arg in arg_list]
    msg = ""
    for a in arg_list:
        if "." in a:
            continue
        if a not in df.columns:
            msg += "{} is required unless {} is present.  ".format(col_name, a)
    if msg:
        return msg
    else:
        return None
    return None


def requiredUnlessTable(col_name, df, arg, *args):
    """
    Col_name must be present in df unless
    arg (table_name) is present in contribution
    """
    table_name = arg
    if col_name in df.columns:
        return None
    elif table_name in current_con.tables:
        return None
    else:
        #print "{} is required unless table {} is present".format(col_name, table_name)
        return "{} is required unless table {} is present".format(col_name, table_name)

    
def requiredIfGroup(col_name, df, arg, *args):
    """
    Col_name is required if other columns of 
    the group arg are present.
    """
    group_name = arg
    groups = set()
    columns = df.columns
    for col in columns:
        if col not in current_dm.index:
            continue
        group = current_dm.loc[col]['group']
        groups.add(group)
    if group_name in groups:
        if col_name in columns:
            return None
        else:
            #print "{} is required if column group {} is used".format(col_name, group_name)
            return "{} is required if column group {} is used".format(col_name, group_name)
    return None


def required(col_name, df, arg):
    if col_name in df.columns:
        return None
    else:
        return "{} is required".format(col_name) 

def isIn(row, col_name, arg, dm, df):
    #grade = df.apply(func, args=(validation_name, arg, dm), axis=1)
    x = 0
    cell_value = row[col_name]
    if not cell_value:
        return None
    # if it's in another table
    cell_values = [v.strip(" ") for v in cell_value.split(":")]
    if "." in arg:
        table_name, table_col_name = arg.split(".")
        if table_name not in current_con.tables:
            return "Must contain a value from {} table. Missing {} table.".format(table_name, table_name)
        if table_col_name not in current_con.tables[table_name].df.columns:
            return "{} table is missing {} column, which is required for validation".format(table_name, table_col_name)
        possible_values = current_con.tables[table_name].df[table_col_name].unique()
        for value in cell_values:
            if value not in possible_values:
                return "This value: {} is not found in: {}".format(value, arg)
                break
    # if it's in the present table:
    else:
        possible_values = df[arg].unique()
        for value in cell_values:
            if value not in possible_values:
                return "This value: {} is not found in: {} column".format(value, arg)
                break
    return None
    
def checkMax(row, col_name, arg, *args):
    cell_value = row[col_name]
    if not cell_value:
        return None
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    #arg = float(arg)
    try:
        if float(cell_value) <= float(arg):
            return None
        else:
            #print "{} must be <= {}".format(str(cell_value), str(arg))
            return "{} must be <= {}".format(str(cell_value), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def checkMin(row, col_name, arg, *args):
    cell_value = row[col_name]
    if not cell_value:
        return None
    try:
        arg = float(arg)
    except ValueError:
        arg = row[arg]
    try:
        if float(cell_value) >= float(arg):
            return None
        else:
            return "{} must be >= {}".format(str(cell_value), str(arg))
    # this happens when the value isn't a float (an error which will be caught elsewhere)
    except ValueError:
        return None

def cv(row, col_name, arg, current_data_model, *args):
    cell_value = row[col_name]
    if not cell_value:
        return None
    if cell_value.lower() in [v.lower() for v in vocabulary[col_name]]:
        return None
    else:
        return "{} is not in {}".format(cell_value, arg)
        

# validate presence
presence_operations = {"required": required, "requiredUnless": requiredUnless,
                       "requiredIfGroup": requiredIfGroup, 
                       'requiredUnlessTable': requiredUnlessTable}
# validate values
value_operations = {"max": checkMax, "min": checkMin, "cv": cv, "in": isIn}

def split_func(string):
    """
    Take a string like 'requiredIf("arg_name")'
    return the function name and the argument:
    (requiredIf, arg_name)
    """
    ind = string.index("(")
    return string[:ind], string[ind+1:-1].strip('"')


def test_type(value, value_type):
    if not value:
        return None
    if value_type == "String":
        if str(value) == value:
            return None
        else:
            return "should be string"
    elif value_type == "Number":
        try:
            float(value)
            return None
        except ValueError:
            return "should be a number"
    elif value_type == "Integer":
        if isinstance(value, str):
            if str(int(value)) == value:
                return None
            else:
                return "should be an integer"
        else:
            if int(value) == value:
                return None
            else:
                return "should be an integer"
    else:
        return None
    #String, Number, Integer, List, Matrix, Dictionary, Text
    


def validate_df(df, dm):
    # check column validity
    cols = df.columns
    invalid_cols = [col for col in cols if col not in dm.index]
    for validation_name, validation in dm.iterrows():
        value_type = validation['type']
        if validation_name in df.columns:
            output = df[validation_name].apply(test_type, args=(value_type,))
            df["type_pass" + "_" + validation_name + "_" + value_type] = output

        val_list = validation['validations']
        if not val_list or isinstance(val_list, float):
            continue
        for num, val in enumerate(val_list):
            func_name, arg = split_func(val)
            if arg == "magic_table_column":
                continue
            # first validate for presence
            if func_name in presence_operations:
                func = presence_operations[func_name]
                grade = func(validation_name, current_df, arg)
                pass_col_name = "presence_pass_" + validation_name + "_" + func.__name__
                df[pass_col_name] = grade
    
            # then validate for correct values
            elif func_name in value_operations:
                func = value_operations[func_name]
                if validation_name in df.columns:
                    grade = df.apply(func, args=(validation_name, arg, dm, df), axis=1)
                    col_name = "value_pass_" + validation_name + "_" + func.__name__
                    if col_name in df.columns:
                        num_range = range(1, 10)
                        for num in num_range:
                            if (col_name + str(num)) in df.columns:
                                continue
                            else:
                                col_name = col_name + str(num)
                                break
                    df[col_name] = grade.astype(object)
    return df

  
    

#for dtype in current_con.tables:
#    print dtype
#    current_df = current_con.tables[dtype].df
#    current_dm = current_con.tables[dtype].data_model.dm[dtype]
#    current_df = validate_df(current_df, current_dm)
    

# check that values pass validation
# validation checks to add:
# sv (suggested vocab)
# requiredOneInGroup
# requiredUnlessSynthetic


# re-do upload_magic to use contribution-level (??)

# first, do validations on each table in the contribution
# this will include removing unneeded data (RmKeys from old upload_magic)
# this will also include checking everything against the data model (strings are strings, etc.)g


# next, splat out each table into a file and wrap it up.  give it a sensible name.  

In [5]:
current_df = current_con.tables['samples'].df  
current_dm = current_con.tables['samples'].data_model.dm['samples']

current_df = validate_df(current_df, current_dm)



In [6]:
current_df.columns
current_dm.head()

,description,examples,group,label,notes,position,previous_columns,type,unit,urls,validations
alteration,"Sample alteration, Grade",NaN,Geology,Alteration,NaN,20,"[{u'column': u'sample_alteration', u'table': u...",String,NaN,NaN,"[cv(""alteration"")]"
alteration_type,"Sample alteration, Type",NaN,Geology,Alteration Type,NaN,21,"[{u'column': u'sample_alteration_type', u'tabl...",String,NaN,NaN,"[cv(""alteration_type"")]"
analysts,Colon-delimited list of EarthRef handles and/o...,[@user1:@user2:Not A. Member <no.earthref.hand...,Metadata,Analyst Names,NaN,101,"[{u'column': u'er_analyst_mail_names', u'table...",List,NaN,NaN,"[type(""users"")]"
aniso_f,"Foliation, T2/T3",NaN,Anisotropy,Anisotropy F,NaN,86,"[{u'column': u'anisotropy_f', u'table': u'rmag...",Number,Dimensionless,NaN,NaN
aniso_ff,"Log foliation, ln(F)",NaN,Anisotropy,Anisotropy F',NaN,88,"[{u'column': u'anisotropy_ff', u'table': u'rma...",Number,Dimensionless,[http://dx.doi.org/10.1130/0016-7606(1977)88<1...,NaN


In [7]:
current_con.tables['specimens'].df.head()

,aniso_alt,aniso_ftest,aniso_ftest12,aniso_ftest23,aniso_n,aniso_s,aniso_s_n_measurements,aniso_s_sigma,aniso_type,citations,...,lithologies,magic_method_codes,meas_step_max,meas_step_min,meas_step_unit,method_codes,result_quality,sample,software_packages,specimen
specimen,,,,,,,,,,,,,,,,,,,,,
hz05a1,2.13,1.173968,1.819236,0.057769,6.000000,0.344050:0.328649:0.327301:-0.009684:-0.000697...,6.000000,0.011113,ATRM,This study,...,Not Specified,AE-H:LP-AN-TRM,None,None,None,AE-H:LP-AN-TRM,None,hz05a,None,hz05a1
hz05a1,None,1.173968,1.819236,0.057769,6.000000,0.344050:0.328649:0.327301:-0.009684:-0.000697...,None,0.011113,None,This study,...,None,AE-H:DA-AC-ATRM:DA-CR:IE-TT:LP-AN-TRM:LP-PI-AL...,803,373,K,AE-H:DA-AC-ATRM:DA-CR:IE-TT:LP-AN-TRM:LP-PI-AL...,None,hz05a,None,hz05a1
hz05a2,0.84,4.222956,4.552863,1.113550,6.000000,0.331962:0.327829:0.340209:-0.001051:0.012687:...,6.000000,0.007775,ATRM,This study,...,Not Specified,AE-H:LP-AN-TRM,None,None,None,AE-H:LP-AN-TRM,None,hz05a,None,hz05a2
hz05a2,None,4.222956,4.552863,1.113550,6.000000,0.331962:0.327829:0.340209:-0.001051:0.012687:...,None,0.007775,None,This study,...,None,AE-H:DA-AC-ATRM:DA-CR:IE-TT:LP-AN-TRM:LP-PI-AL...,833,473,K,AE-H:DA-AC-ATRM:DA-CR:IE-TT:LP-AN-TRM:LP-PI-AL...,None,hz05a,None,hz05a2
hz05a3,0.58,3.586323,3.979692,0.876838,6.000000,0.330940:0.332237:0.336823:0.013169:-0.006450:...,6.000000,0.008471,ATRM,This study,...,Not Specified,AE-H:LP-AN-TRM,None,None,None,AE-H:LP-AN-TRM,None,hz05a,None,hz05a3


In [8]:

value_cols = current_df.columns.str.match("^value_pass_")
present_cols = current_df.columns.str.match("^presence_pass")
type_cols = current_df.columns.str.match("^type_pass_")

value_col_names = current_df.columns[value_cols]
present_col_names = current_df.columns[present_cols]
type_col_names = current_df.columns[type_cols]

validation_cols = np.where(value_cols, value_cols, present_cols)
validation_cols = np.where(validation_cols, validation_cols, type_cols)
validation_col_names = current_df.columns[validation_cols]
current_df[type_col_names].dropna(how='all', axis=1).head()

,type_pass_lat_Number
sample,
hz05a,None
hz05a,should be a number
hz05b,None
hz05b,None
hz05c,None


In [9]:
current_df[present_col_names].dropna(how='all', axis=1).head()


,presence_pass_citations_required,presence_pass_result_quality_required
sample,,
hz05a,citations is required,result_quality is required
hz05a,citations is required,result_quality is required
hz05b,citations is required,result_quality is required
hz05b,citations is required,result_quality is required
hz05c,citations is required,result_quality is required


In [10]:
current_df[value_col_names].dropna(how='all', axis=1).head()


,value_pass_lon_checkMax,value_pass_site_isIn,value_pass_specimens_isIn
sample,,,
hz05a,600.0 must be <= 360.0,None,None
hz05a,None,None,None
hz05b,None,None,This value: fake is not found in: specimens.sp...
hz05b,None,None,This value: fake is not found in: specimens.sp...
hz05c,None,None,None


In [11]:
#  return all: 
#    rows with a problem
#    columns with a problem
#    cells with a problem
#    missing columns
#    extra columns




In [12]:
s = current_df.iloc[0]
dir(s[s.notnull()])
current_df.head()

,age,age_high,age_low,age_unit,azimuth,cooling_rate,criteria,description,dip,geologic_classes,...,type_pass_sample_String,presence_pass_sample_required,type_pass_site_String,value_pass_site_isIn,presence_pass_site_required,type_pass_software_packages_List,type_pass_specimens_List,value_pass_specimens_isIn,presence_pass_vgp_lat_requiredIfGroup,presence_pass_vgp_lon_requiredIfGroup
sample,,,,,,,,,,,,,,,,,,,,,
hz05a,None,None,None,None,None,4.38E+11,None,paleointensity mean,None,Archeologic,...,None,None,None,None,None,None,None,None,None,None
hz05a,-740,-732,-800,Years Cal AD (+/-),None,None,ACCEPT,Paleointensity; V[A]DM,None,None,...,None,None,None,None,None,None,None,None,None,None
hz05b,None,None,None,None,None,4.38E+11,None,paleointensity mean,None,Archeologic,...,None,None,None,None,None,None,None,This value: fake is not found in: specimens.sp...,None,None
hz05b,-740,-732,-800,Years Cal AD (+/-),None,None,ACCEPT,Paleointensity; V[A]DM,None,None,...,None,None,None,None,None,None,None,This value: fake is not found in: specimens.sp...,None,None
hz05c,None,None,None,None,None,4.38E+11,None,paleointensity mean,None,Archeologic,...,None,None,None,None,None,None,None,None,None,None


In [21]:
def check_row(row):
    ind = row[row.notnull()].index
    values = row[row.notnull()].values
    return str(zip(ind, values))


print "problem rows (value & type problems)"
current_df["num"] = range(len(current_df))
# get column names for value & type validations
names = value_col_names.union(type_col_names)
# drop all non validation columns
value_problems = current_df[names.union(["num"])]
failing_items = value_problems.dropna(how="all", subset=names)
failing_items = failing_items.dropna(how="all", axis=1)
# get names of the failing items
bad_items = list(failing_items.index)
# get index numbers of the failing items
bad_indices = list(failing_items["num"])
zip(bad_indices, bad_items)
failing_items['issues'] = failing_items.drop("num", axis=1).apply(check_row, axis=1).values
failing_items["issues"]

problem rows (value & type problems)


sample
hz05a        [(u'value_pass_lon_checkMax', '600.0 must be <...
hz05a        [(u'type_pass_lat_Number', 'should be a number')]
hz05b        [(u'value_pass_specimens_isIn', 'This value: f...
hz05b        [(u'value_pass_specimens_isIn', 'This value: f...
hz05c        [(u'value_pass_specimens_isIn', 'This value: f...
hz05e        [(u'value_pass_site_isIn', 'This value: fake_s...
mgh03k       [(u'value_pass_site_isIn', 'This value: mgh05 ...
mgh05a       [(u'value_pass_site_isIn', 'This value: mgh05 ...
mgh05b       [(u'value_pass_site_isIn', 'This value: mgh05 ...
mgh05c       [(u'value_pass_site_isIn', 'This value: mgh05 ...
mgh05d       [(u'value_pass_site_isIn', 'This value: mgh05 ...
mgh05e       [(u'value_pass_site_isIn', 'This value: mgh05 ...
mgh05f       [(u'value_pass_site_isIn', 'This value: mgh05 ...
mgh12t101    [(u'value_pass_site_isIn', 'This value: mgh12t...
mgh12t102    [(u'value_pass_site_isIn', 'This value: mgh12t...
mgh12t1a1    [(u'value_pass_site_isIn', 'This va

In [14]:
# add in numbered index
current_df['num'] = range(len(current_df))
# rows with problems:
print 'Problem rows (all problems):'

problems = current_df[validation_col_names.union(["num"])]
problems = problems.dropna(how='all', axis=0, subset=validation_col_names)
problems = problems.dropna(how='all', axis=1)
zip(list(problems["num"]), list(problems.index))[:10]

Problem rows (all problems):


[(0, 'hz05a'),
 (1, 'hz05a'),
 (2, 'hz05b'),
 (3, 'hz05b'),
 (4, 'hz05c'),
 (5, 'hz05c'),
 (6, 'hz05e'),
 (7, 'hz05e'),
 (8, 'hz05f'),
 (9, 'hz05f')]

In [15]:

#columns with problems (of ANY type)
bad_cols = problems.columns
prefixes = ["presence_pass_", "value_pass_", "type_pass_"]
prefixes = ["value_pass_", "type_pass_"]
missing_prefix = "presence_pass_"
problem_cols = []
for col in bad_cols:
    end = col.rfind("_")
    for prefix in prefixes:
        if col.startswith(prefix):
            problem_cols.append(col[len(prefix):end])
            continue
print 'Problem columns (wrong type or invalid value):'
print problem_cols

missing_cols = []
for col in bad_cols:
    end = col.rfind("_")
    if col.startswith(missing_prefix):
        missing_cols.append(col[len(missing_prefix):end])
        continue

print 'Missing columns:'
print missing_cols

Problem columns (wrong type or invalid value):
[u'lat', u'lon', u'site', u'specimens']
Missing columns:
[u'citations', u'result_quality']


In [16]:
value_problems.dropna(how="all", axis=1)

,num,type_pass_lat_Number,value_pass_lon_checkMax,value_pass_site_isIn,value_pass_specimens_isIn
sample,,,,,
hz05a,0,None,600.0 must be <= 360.0,None,None
hz05a,1,should be a number,None,None,None
hz05b,2,None,None,None,This value: fake is not found in: specimens.sp...
hz05b,3,None,None,None,This value: fake is not found in: specimens.sp...
hz05c,4,None,None,None,None
hz05c,5,None,None,None,This value: fake_specimen is not found in: spe...
hz05e,6,None,None,None,None
hz05e,7,None,None,This value: fake_site is not found in: sites.site,None
hz05f,8,None,None,None,None


In [17]:
current_df.iloc[problems['num']]
len(current_df)
len(problems['num'])

244

## Filling in an existing dataframe

In [18]:
# keep all of df1, add in any extra from df2
df1 = pd.DataFrame(np.random.randint(1, 10, (3, 5)), columns=['one', 'two', 'three', 'four', 'five'])
df1.iloc[0, 1] = np.nan
df1.iloc[2, 2] = np.nan
df2 = pd.DataFrame(np.random.randint(1, 10, (3, 5)), columns=['one', 'three', 'five', 'seven', 'nine'])
df1

,one,two,three,four,five
0,1,NaN,2.0,1,2
1,8,9.0,7.0,4,2
2,2,1.0,NaN,6,5


In [19]:
df2

,one,three,five,seven,nine
0,6,2,9,5,1
1,4,3,8,6,3
2,2,2,1,7,3


In [20]:
unique_df2_cols = df2.columns.difference(df1.columns)
unique_df2 = df2[unique_df2_cols]

# this adds in all the unique columns that weren't in df1
concat_df = pd.concat([df1, unique_df2], axis=1)
# fills in null values in df1 with values from df2
concat_df.fillna(df2)

,one,two,three,four,five,nine,seven
0,1,NaN,2.0,1,2,1,5
1,8,9.0,7.0,4,2,3,6
2,2,1.0,2.0,6,5,3,7
